<a href="https://colab.research.google.com/github/WhackyBoyg0d/QuantMomentumTrading/blob/main/Momentum_Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Quantative Momentum Strategy**

Investing in stocks that have increased in price the most. 
50 stocks with the highest price momentum will be selected.

In [5]:
!pip install XlsxWriter

import pandas as pd
import numpy as np
import requests
import math
from scipy import stats
import xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 6.8 MB/s eta 0:00:00


In [6]:
from google.colab import files
files.upload()

Saving sp_500_stocks.csv to sp_500_stocks.csv


{'sp_500_stocks.csv': b'Ticker\r\nA\r\nAAL\r\nAAP\r\nAAPL\r\nABBV\r\nABC\r\nABMD\r\nABT\r\nACN\r\nADBE\r\nADI\r\nADM\r\nADP\r\nADSK\r\nAEE\r\nAEP\r\nAES\r\nAFL\r\nAIG\r\nAIV\r\nAIZ\r\nAJG\r\nAKAM\r\nALB\r\nALGN\r\nALK\r\nALL\r\nALLE\r\nALXN\r\nAMAT\r\nAMCR\r\nAMD\r\nAME\r\nAMGN\r\nAMP\r\nAMT\r\nAMZN\r\nANET\r\nANSS\r\nANTM\r\nAON\r\nAOS\r\nAPA\r\nAPD\r\nAPH\r\nAPTV\r\nARE\r\nATO\r\nATVI\r\nAVB\r\nAVGO\r\nAVY\r\nAWK\r\nAXP\r\nAZO\r\nBA\r\nBAC\r\nBAX\r\nBBY\r\nBDX\r\nBEN\r\nBF.B\r\nBIIB\r\nBIO\r\nBK\r\nBKNG\r\nBKR\r\nBLK\r\nBLL\r\nBMY\r\nBR\r\nBRK.B\r\nBSX\r\nBWA\r\nBXP\r\nC\r\nCAG\r\nCAH\r\nCARR\r\nCAT\r\nCB\r\nCBOE\r\nCBRE\r\nCCI\r\nCCL\r\nCDNS\r\nCDW\r\nCE\r\nCERN\r\nCF\r\nCFG\r\nCHD\r\nCHRW\r\nCHTR\r\nCI\r\nCINF\r\nCL\r\nCLX\r\nCMA\r\nCMCSA\r\nCME\r\nCMG\r\nCMI\r\nCMS\r\nCNC\r\nCNP\r\nCOF\r\nCOG\r\nCOO\r\nCOP\r\nCOST\r\nCOTY\r\nCPB\r\nCPRT\r\nCRM\r\nCSCO\r\nCSX\r\nCTAS\r\nCTL\r\nCTSH\r\nCTVA\r\nCTXS\r\nCVS\r\nCVX\r\nCXO\r\nD\r\nDAL\r\nDD\r\nDE\r\nDFS\r\nDG\r\nDGX\r\nDHI\r\nDHR\r\nDIS

In [7]:
# Importing list of stocks

stocks = pd.read_csv('sp_500_stocks.csv')

In [8]:
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [9]:
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


In [11]:
# Testing if the API is working
IEX_CLOUD_API_TOKEN = 'sk_78880833672e4010b46b8e989493af76'
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'

data = requests.get(api_url)
print(data.json())

{'companyName': 'Apple Inc', 'marketcap': 2846108626900, 'week52high': 181.78, 'week52low': 123.64, 'week52highSplitAdjustOnly': 181.78, 'week52lowSplitAdjustOnly': 124.17, 'week52change': 0.25541591072289593, 'sharesOutstanding': 15728702000, 'float': 0, 'avg10Volume': 71622103, 'avg30Volume': 54752734, 'day200MovingAvg': 152.8, 'day50MovingAvg': 171, 'employees': 164000, 'ttmEPS': 5.88, 'ttmDividendRate': 0.9264396314120724, 'dividendYield': 0.005119865329715792, 'nextDividendDate': '', 'exDividendDate': '2023-05-12', 'nextEarningsDate': '2023-08-03', 'peRatio': 30.17470793248587, 'beta': 0, 'maxChangePercent': 69.44967880085653, 'year5ChangePercent': 2.9585830919607616, 'year2ChangePercent': 0.45805953588663617, 'year1ChangePercent': 0.25541591072289593, 'ytdChangePercent': 0.39866031607636176, 'month6ChangePercent': 0.23102188763038975, 'month3ChangePercent': 0.2014227182097705, 'month1ChangePercent': 0.09446248555182502, 'day30ChangePercent': 0.0454050706334479, 'day5ChangePercent

In [12]:
# Parsing the call
data = data.json()
data['year1ChangePercent']

0.25541591072289593

In [13]:
# Executing batch API call and building DataFrame

def chunks(lst, n):
    """Yield succesive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [14]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = [ 'Ticker', 'Stock Price', 'One-Year Price Return', 'Number of Shares to Buy']

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url =  f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        s1 = pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A'
            ], index = my_columns)
        final_dataframe = final_dataframe.append(s1, 
                                                ignore_index=True)
final_dataframe

<ipython-input-14-3074aee3cbfb>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(s1,
<ipython-input-14-3074aee3cbfb>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(s1,
<ipython-input-14-3074aee3cbfb>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(s1,
<ipython-input-14-3074aee3cbfb>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(s1,
<ipython-input-14-3074aee3cbfb>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

,Ticker,Stock Price,One-Year Price Return,Number of Shares to Buy
0,A,118.22,-0.073820,N/A
1,AAL,14.87,-0.083231,N/A
2,AAP,67.56,-0.631121,N/A
3,AAPL,180.95,0.255416,N/A
4,ABBV,136.87,-0.033916,N/A
...,...,...,...,...
95,CINF,100.58,-0.182675,N/A
96,CL,76.05,-0.011808,N/A
97,CLX,161.37,0.199761,N/A
98,CMA,40.93,-0.477837,N/A


In [15]:
# Removing Low-Momentum stocks

final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe_top = final_dataframe[:50]
final_dataframe_top.reset_index(inplace = True)
final_dataframe_top 


,index,Ticker,Stock Price,One-Year Price Return,Number of Shares to Buy
0,77,CAH,84.59,0.618960,N/A
1,37,ANET,162.50,0.575070,N/A
2,55,BA,213.32,0.531921,N/A
3,62,BIIB,300.20,0.496361,N/A
4,50,AVGO,812.00,0.476218,N/A
5,85,CDNS,229.69,0.448600,N/A
6,19,AIV,8.66,0.386399,N/A
7,72,BSX,51.82,0.298421,N/A
8,21,AJG,205.68,0.282088,N/A
9,39,ANTM,482.58,0.281524,N/A


In [16]:
for i in range(len(final_dataframe)):
  if(final_dataframe.iloc[i]["Ticker"]=="ANET"):
    print(final_dataframe.iloc[i])

Ticker                        ANET
Stock Price                  162.5
One-Year Price Return      0.57507
Number of Shares to Buy        N/A
Name: 37, dtype: object


In [17]:
# Calculating the Number of Shares to Buy

def portfolio_input():
  global portfolio_size
  portfolio_size = input("Enter the size of portfolio")
  try:
    val = float(portfolio_size)
  except ValueError:
    print('Please enter an integer')
    portfolio_size = float(input('Enter Value of Your portfolio: '))

portfolio_input()
print(portfolio_size)

Enter the size of portfolio
Please enter an integer
Enter Value of Your portfolio: 1000000
1000000.0


In [20]:
position_size = float(portfolio_size)/len(final_dataframe_top.index)
print(position_size)
for i in range(0, len(final_dataframe_top.index)):
    final_dataframe_top.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe_top.loc[i, 'Stock Price'])
    
final_dataframe_top   

20000.0


,index,Ticker,Stock Price,One-Year Price Return,Number of Shares to Buy
0,77,CAH,84.59,0.618960,236
1,37,ANET,162.50,0.575070,123
2,55,BA,213.32,0.531921,93
3,62,BIIB,300.20,0.496361,66
4,50,AVGO,812.00,0.476218,24
5,85,CDNS,229.69,0.448600,87
6,19,AIV,8.66,0.386399,2309
7,72,BSX,51.82,0.298421,385
8,21,AJG,205.68,0.282088,97
9,39,ANTM,482.58,0.281524,41
